<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [14]:
import os
import re
import sys
import numpy as np
import pandas as pd

from time import sleep, time
from keras import backend as K
from keras.models import Model 
from keras.models import Sequential as SequentialModel
from keras.models import load_model
from keras.layers import Dense, Conv1D, LSTM, Dropout, Embedding, Layer, Input, Flatten, concatenate as Concatenate, Lambda, Add
from keras.callbacks import Callback
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer as KerasTokenizer
from tqdm.notebook import tqdm
from sometimer import timer, time_this_method

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

sys.path.insert(0, '../ct')

import load
from preprocess import preprocess
from preprocess import Tokenizer
from preprocess.preprocess import separator_samples

from model.layers import LayerNormalization
from model.layers import ContentBasedAttention_CT
from model.layers import ScaledDotProductAttention
from model.layers import MultiHeadAttention
from model.layers import content_based_attention
from model.layers import ReverseEmbedding

from model import CompressiveTransformer
from model import AttentionReconstruction

from model.callbacks import ClearCompressedMemory, WriteLogsToFile
from train.generators import next_token_batch_generator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
x = Input(shape=(10,))
emb = Embedding(input_dim=100,
                output_dim=20)
e = emb(x)

den = Dense(units=5, 
            activation='relu',
            use_bias=False)
d = den(e)
d2 = Dense(units=20)(d)

o = ReverseEmbedding(embedding_layer=emb)(d2)

model = Model(inputs=[x], outputs=[o])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

input_emb.shape=(None, 20)
w_tanspose.shape=(20, 100)
Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 10)                0         
_________________________________________________________________
embedding_15 (Embedding)     (None, 10, 20)            2000      
_________________________________________________________________
dense_29 (Dense)             (None, 10, 5)             100       
_________________________________________________________________
dense_30 (Dense)             (None, 10, 20)            120       
_________________________________________________________________
reverse_embedding_11 (Revers (None, 100)               2000      
Total params: 4,220
Trainable params: 2,220
Non-trainable params: 2,000
_________________________________________________________________


In [88]:
model.fit(x=np.zeros((20, 10)),
          y=np.zeros((20, 100)),
          batch_size=10)

/home/vs/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
20/20 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


<tf.Tensor: shape=(4, 100), dtype=float32, numpy=
array([[-3.12270343e-01, -2.01881075e+00,  1.10959005e+00,
         4.12544519e-01, -2.48004246e+00, -1.80119312e+00,
         9.10758972e-05,  3.86740834e-01,  2.00239515e+00,
         4.82137024e-01,  2.14934325e+00, -1.36382669e-01,
         4.13869888e-01,  6.01750076e-01, -1.75808585e+00,
        -2.99215984e+00, -1.49753630e-01,  8.09358835e-01,
         1.13806009e+00, -1.11401045e+00,  1.08204746e+00,
        -1.15455687e+00, -1.19104409e+00, -2.34749508e+00,
        -2.36000752e+00,  5.59672356e-01, -3.97789180e-01,
         1.11688316e+00,  3.40092182e-02, -2.31264973e+00,
        -2.44603539e+00,  3.86489809e-01, -2.06056976e+00,
        -5.19997478e-01, -3.78525734e-01,  1.51727593e+00,
        -3.59310627e-01,  4.82630014e-01, -1.92343384e-01,
         1.51048589e+00,  5.14758825e-01, -1.21477628e+00,
        -2.15410650e-01,  5.64563870e-01, -3.14754844e-02,
         9.07374620e-02, -3.32142019e+00,  7.28994787e-01,
      

In [75]:
emb.embeddings

<tf.Variable 'embedding_9/embeddings:0' shape=(100, 20) dtype=float32, numpy=
array([[-0.02769239,  0.04272921, -0.01580853, ...,  0.01696021,
        -0.03627395, -0.03013293],
       [-0.0397629 ,  0.04580368,  0.02049183, ..., -0.00394722,
        -0.03664054,  0.02503315],
       [-0.01037703,  0.00062399, -0.02629734, ...,  0.00161244,
        -0.02797332, -0.02967653],
       ...,
       [ 0.02804993,  0.04950825,  0.03924369, ...,  0.04061778,
        -0.03057084,  0.01604346],
       [ 0.02938082, -0.0223686 ,  0.00598091, ...,  0.01112235,
        -0.04460507, -0.04773163],
       [-0.00549177,  0.04360482, -0.03629843, ..., -0.04787669,
         0.01324466, -0.03822105]], dtype=float32)>